In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("merged_data.csv")  # Replace with actual file path

# Extract relevant data
df_gdp = df[["year", "gdp"]].copy()

# Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
df_gdp.loc[:, "scaled_gdp"] = scaler.fit_transform(df_gdp[["gdp"]])

# Prepare sequences for LSTM
def create_sequences(data, seq_length=10):
    sequences, labels = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

seq_length = 10
X, y = create_sequences(df_gdp["scaled_gdp"].values, seq_length)

# Convert to PyTorch tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Training LSTM model
model = LSTMModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(-1).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(-1).to(device)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()

# Predicting future GDP for years 111 to 115
future_years = list(range(111, 116))
future_inputs = df_gdp["scaled_gdp"].values[-seq_length:].reshape(1, seq_length, 1)
future_predictions = []

model.eval()
with torch.no_grad():
    for year in future_years:
        pred = model(torch.tensor(future_inputs, dtype=torch.float32).to(device)).cpu().numpy()
        future_predictions.append(pred[0, 0])
        future_inputs = np.roll(future_inputs, -1, axis=1)
        future_inputs[0, -1, 0] = pred[0, 0]

# Convert predictions back to original scale
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1)).flatten()

# Apply policy changes
for i, year in enumerate(future_years):
    if year in [111, 114]:  # Sturgeons boost fishermen's income, trade agreements raise GDP
        future_predictions[i] *= 1.03  # Trade agreement +3% GDP increase
    if year in [112, 115]:  # Locusts impact farmers' output
        future_predictions[i] *= 0.97  # 3% GDP reduction due to agricultural impact

# Store predictions in DataFrame
predictions_df = pd.DataFrame({"year": future_years, "predicted_gdp": future_predictions})

# Save predictions to CSV
predictions_df.to_csv("predicted_gdp_111_115_policy_changes.csv", index=False)
print("Predictions saved to predicted_gdp_111_115_policy_changes.csv")

# Plot results
plt.figure(figsize=(10,5))
plt.plot(df_gdp["year"], df_gdp["gdp"], label="Actual GDP")
plt.plot(future_years, future_predictions, label="Predicted GDP (Policy Adjusted)", linestyle="dashed")
plt.xlabel("Year")
plt.ylabel("GDP")
plt.legend()
plt.title("LSTM GDP Predictions with Policy Changes for Years 111-115")
plt.show()